In [0]:
%tensorflow_version 2.x
import tensorflow as tf

import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc, roc_curve
from sklearn.metrics import classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [47]:
from google.colab import drive
drive.mount(('/content/drive'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bank.csv')

In [49]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
df.shape

(10000, 14)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [0]:
df = df.set_index('RowNumber')

In [53]:
df['Surname'].value_counts()

Smith         32
Scott         29
Martin        29
Walker        28
Brown         26
              ..
Fielding       1
Yirawala       1
Titheradge     1
Bogdanova      1
Candler        1
Name: Surname, Length: 2932, dtype: int64

Dropping surname and customer ID

In [0]:
df.drop(['CustomerId', 'Surname'],axis=1, inplace = True)

In [55]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

There are no null values

In [56]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
CreditScore,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
Age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48
Exited,10000.0,0.203700,0.402769,0.00,0.00,0.000,0.0000,1.00


In [57]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

The number of rows for an exited customer is extremely low compared to that of an non-exited customer. We will be using upsampling on the training data 

In [58]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [59]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

We will be converting the two object columns using pd.get_dummies

In [0]:
df = pd.get_dummies(df)

In [0]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)

In [63]:
sc = StandardScaler()
#temp = X_train[['Surname', 'Geography', 'Gender']]
#X_train = X_train.drop(['Surname', 'Geography', 'Gender'], axis=1)
X_train[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']] = sc.fit_transform(X_train[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [64]:
X_test[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']] = sc.fit_transform(X_test[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [65]:
y_train.value_counts()

0    5960
1    1540
Name: Exited, dtype: int64

Using smote to upsample

In [0]:
sm = SMOTE(random_state=42, ratio=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [67]:
pd.value_counts(y_train)

1    5960
0    5960
dtype: int64

Now both classes are equally represented

Building the model

In [0]:
model = tf.keras.models.Sequential()

In [0]:
n_cols = X_train.shape[1]

In [0]:
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [72]:
model.fit(X_train,y_train, batch_size=10, epochs=100)

Train on 11920 samples
Epoch 1/100
11920/11920 [==============================] - 3s 231us/sample - loss: 0.5564 - accuracy: 0.7190
Epoch 2/100
11920/11920 [==============================] - 3s 210us/sample - loss: 0.4704 - accuracy: 0.7754
Epoch 3/100
11920/11920 [==============================] - 2s 208us/sample - loss: 0.4597 - accuracy: 0.7822
Epoch 4/100
11920/11920 [==============================] - 2s 207us/sample - loss: 0.4548 - accuracy: 0.7856
Epoch 5/100
11920/11920 [==============================] - 2s 208us/sample - loss: 0.4511 - accuracy: 0.7855
Epoch 6/100
11920/11920 [==============================] - 3s 216us/sample - loss: 0.4482 - accuracy: 0.7881
Epoch 7/100
11920/11920 [==============================] - 3s 211us/sample - loss: 0.4454 - accuracy: 0.7898
Epoch 8/100
11920/11920 [==============================] - 3s 214us/sample - loss: 0.4435 - accuracy: 0.7922
Epoch 9/100
11920/11920 [==============================] - 2s 210us/sample - loss: 0.4417 - accuracy: 0.7

Checking loss and accuracy of the model

In [73]:
eval_model=model.evaluate(X_train, y_train)
eval_model

11920/11920 [==============================] - 1s 62us/sample - loss: 0.4029 - accuracy: 0.8149


[0.40292282032486576, 0.8149329]

Predicting using the model

In [0]:
y_pred=model.predict(X_test)
y_pred =(y_pred>0.5)

In [75]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.83      0.87      2003
           1       0.51      0.72      0.59       497

    accuracy                           0.81      2500
   macro avg       0.71      0.77      0.73      2500
weighted avg       0.84      0.81      0.82      2500



In [76]:
print(confusion_matrix(y_test, y_pred))

[[1656  347]
 [ 140  357]]


Optimizing model

In [0]:
def algorithm_pipeline(X_train, X_test, y_train, y_test, 
                       model, param_grid, cv=10, scoring_fit='accuracy',
                       do_probabilities = True):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train, y_train)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test)
    else:
      pred = fitted_model.predict(X_test)
    
    return fitted_model, pred

In [0]:
def build_cnn(activation = 'relu',
              dropout_rate = 0.2,
              optimizer = 'Adam'):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(10, activation=activation, input_shape=(n_cols,)))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(10, activation=activation))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model

In [0]:
param_grid = {
              'batch_size':[64,128],
              'epochs' :              [75,100],
              'optimizer' :           ['Adam', 'Nadam'],
              'dropout_rate' :        [0.2, 0.3,0.5],
              'activation' :          ['relu', 'elu']
             }

In [0]:
model = KerasClassifier(build_fn = build_cnn, verbose=0)

In [39]:
model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                        param_grid, cv=5, scoring_fit='accuracy')

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 15.1min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 57.6min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 83.6min finished


In [40]:
print(model.best_score_)
print(model.best_params_)

0.7270973154362416
{'activation': 'elu', 'batch_size': 128, 'dropout_rate': 0.2, 'epochs': 100, 'optimizer': 'Adam'}


Running at a low batch size results in memory loss while gridsearchCV so building the new model which gives a slightly lower accuracy but at a faster rate

Building a model with best results

In [0]:
model_final = tf.keras.models.Sequential()
n_cols = X_train.shape[1]
model_final.add(tf.keras.layers.Dense(10, activation='elu', input_shape=(n_cols,)))
model_final.add(tf.keras.layers.Dropout(0.2))
model_final.add(tf.keras.layers.Dense(10, activation='elu'))
model_final.add(tf.keras.layers.Dropout(0.2))
model_final.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_final.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [85]:
model_final.fit(X_train,y_train, batch_size=128, epochs=100)

Train on 11920 samples
Epoch 1/100
11920/11920 [==============================] - 1s 53us/sample - loss: 0.6906 - accuracy: 0.5877
Epoch 2/100
11920/11920 [==============================] - 0s 22us/sample - loss: 0.6124 - accuracy: 0.6648
Epoch 3/100
11920/11920 [==============================] - 0s 23us/sample - loss: 0.5962 - accuracy: 0.6837
Epoch 4/100
11920/11920 [==============================] - 0s 27us/sample - loss: 0.5847 - accuracy: 0.7001
Epoch 5/100
11920/11920 [==============================] - 0s 24us/sample - loss: 0.5751 - accuracy: 0.7046
Epoch 6/100
11920/11920 [==============================] - 0s 23us/sample - loss: 0.5601 - accuracy: 0.7211
Epoch 7/100
11920/11920 [==============================] - 0s 22us/sample - loss: 0.5551 - accuracy: 0.7281
Epoch 8/100
11920/11920 [==============================] - 0s 24us/sample - loss: 0.5495 - accuracy: 0.7247
Epoch 9/100
11920/11920 [==============================] - 0s 22us/sample - loss: 0.5438 - accuracy: 0.7299
Epoch

In [0]:
y_pred=model_final.predict(X_test)
y_pred =(y_pred>0.5)

In [87]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.79      0.85      2003
           1       0.47      0.75      0.58       497

    accuracy                           0.78      2500
   macro avg       0.70      0.77      0.71      2500
weighted avg       0.84      0.78      0.80      2500



The recall result of the optimized model is consistent for both the classes at 79 and 75%. <br>The model can predict approximately 75% of the people who would leave the bank and of the people which the model predicted will leave only 47% will actually leave.